# Improve speed of the app 

In [1]:
import time
import osmnx as ox


import sys
sys.path.append('../../Multimodal_freight_USA/')
from mfreight.Multimodal.graph_utils import MultimodalNet
from mfreight.utils import build_graph

departure_x, departure_y = (30.439440, -85.057166)
arrival_x, arrival_y = (25.382380, -80.475159)

In [3]:
start = time.time()
Net = MultimodalNet(path_u= "../mfreight/Multimodal/data/multimodal_G_tot_u_w_price.plk")
time_data.update({'load_graph':time.time()-start})
print(f'time elapsed: {time.time()-start}s')

time elapsed: 5.008050918579102s


In [11]:
import networkx as nx

In [12]:
a = nx.Graph()
b = nx.MultiDiGraph()

In [16]:
b.is_multigraph()

True

In [10]:
[d for n, d in Net.G_multimodal_u.nodes(data=True) if d['trans_mode'] == 'rail']

[{'trans_mode': 'rail',
  'x': -122.759,
  'y': 45.6385,
  'geometry': <shapely.geometry.point.Point at 0x7fa4446fc070>,
  'key': 0.0,
  'FRANODEID': 303666.0,
  'STATE': 'OR',
  'STCYFIPS': '41051'},
 {'trans_mode': 'rail',
  'x': -122.741,
  'y': 45.6305,
  'geometry': <shapely.geometry.point.Point at 0x7fa4446fc190>,
  'key': 0.0,
  'FRANODEID': 303822.0,
  'STATE': 'OR',
  'STCYFIPS': '41051'},
 {'trans_mode': 'rail',
  'x': -122.747,
  'y': 45.565,
  'geometry': <shapely.geometry.point.Point at 0x7fa4446fc2b0>,
  'key': 0.0,
  'FRANODEID': 303769.0,
  'STATE': 'OR',
  'STCYFIPS': '41051'},
 {'trans_mode': 'rail',
  'x': -122.668,
  'y': 45.5281,
  'geometry': <shapely.geometry.point.Point at 0x7fa4446fc3d0>,
  'key': 0.0,
  'FRANODEID': 304871.0,
  'STATE': 'OR',
  'STCYFIPS': '41051'},
 {'trans_mode': 'rail',
  'x': -122.636,
  'y': 45.459,
  'geometry': <shapely.geometry.point.Point at 0x7fa4446fc4f0>,
  'key': 0.0,
  'FRANODEID': 305065.0,
  'STATE': 'OR',
  'STCYFIPS': '41005'

In [4]:
start = time.time()
Net.set_target_weight_to_edges(0.33, 0.33, 0.33)
print(f'time elapsed: {time.time()-start}s')

time elapsed: 0.3827991485595703s


In [5]:
start = time.time()
n,e = Net.chose_operator(['CSTX',])
print(f'time elapsed: {time.time()-start}s')

time elapsed: 10.451786994934082s
time elapsed: 11.240393161773682s


In [10]:
start = time.time()
build_graph.graph_from_gdfs_revisited(n, e)
print(f'time elapsed: {time.time()-start}s')

time elapsed: 10.139746189117432s


In [9]:
e.head()

,trans_mode,length,duration_h,CO2_eq_kg,geometry,price,target_feature,u,v,key
0,road,1.0,0.000011,0.000081,"LINESTRING (-117.80650 34.06510, -117.80650 34...",0.001044,0.000003,1000502025,1000502025,0
1,road,10.0,0.000112,0.000805,"LINESTRING (-117.80650 34.06510, -117.80640 34...",0.010440,0.000029,1000502025,1000502018,0
2,road,3056.0,0.034337,0.246048,"LINESTRING (-117.83950 34.06870, -117.83910 34...",3.190464,0.008976,1000502025,1000501783,0
3,road,10.0,0.000112,0.000805,"LINESTRING (-117.80640 34.06510, -117.80650 34...",0.010440,0.000029,1000502018,1000502025,0
4,road,342.0,0.003843,0.027535,"LINESTRING (-117.80640 34.06510, -117.80310 34...",0.357048,0.001005,1000502018,1000499705,0


The bottelneck of the app is the `graph_from_gdfs` function. It will be attempted to speed it up

# Improve graph_from_gdfs speed

In [9]:
nodes = Net.nodes.copy()
nodes.drop(
    nodes.columns.difference(
        [
            "trans_mode",
            "x",
            "y",
            "geometry",
            "key"
        ]
    ),
    axis=1,
    inplace=True,
)
nodes.shape

(97087, 5)

In [10]:
edges = Net.edges.copy()
edges.drop(
    edges.columns.difference(
        [
            "trans_mode",
            "length",
            "duration_h",
            "CO2_eq_kg",
            "price",
            "geometry",
            "key",
            "target_feature",
            "u",
            "v",
        ]
    ),
    axis=1,
    inplace=True,
)
edges.shape

(274361, 10)

In [11]:
start = time.time()
G1 = build_graph.graph_from_gdfs_revisited(nodes, edges)
print(f'time elapsed: {time.time()-start}s')

time elapsed: 9.100024938583374s


In [12]:
start = time.time()
G2 = ox.graph_from_gdfs(nodes, edges)
print(f'time elapsed: {time.time()-start}s')

time elapsed: 31.360657215118408s


The proposed improvement is significantly faster.

In [18]:
import networkx as nx
nx.info(G1)

'Name: \nType: MultiDiGraph\nNumber of nodes: 97087\nNumber of edges: 274361\nAverage in degree:   2.8259\nAverage out degree:   2.8259'

In [19]:
nx.info(G2)

'Name: \nType: MultiDiGraph\nNumber of nodes: 97087\nNumber of edges: 274361\nAverage in degree:   2.8259\nAverage out degree:   2.8259'

In [21]:
e1 = ox.graph_to_gdfs(G1, nodes=False)
e1.shape

(274361, 10)

In [22]:
e2 = ox.graph_to_gdfs(G2, nodes=False)
e2.shape

(274361, 10)

In [ ]:

start = time.time()
G = ox.graph_from_gdfs(Net.nodes, Net.edges)
print(f'time elapsed: {time.time()-start}s')